<a href="https://colab.research.google.com/github/aioont/KH012-FinWise/blob/main/FinWise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install tensorflow
!pip install keras

In [12]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#for debug
from IPython.display import clear_output
from google.colab import files

Data Preprocessing

In [ ]:
#read file
df = pd.read_csv("/content/nsestockhistoricalratios.csv")

#Median Imputation
def fill_with_median(column):
  try:
    column_median = column.median(skipna=True)
  except (ValueError, TypeError):
    column_median = 0.5
  return column.fillna(column_median)

for col in df.columns:
    df[col] = fill_with_median(df[col])

#Mean Imputation
def fill_with_mean(column):
  try:
    mean = column.mean(skipna=True)
  except (RuntimeWarning):
    mean = 0.5
  return column.fillna(mean)

for col in df.columns:
    df[col] = fill_with_mean(df[col])

#Deleteing rows with missing values
df.dropna(axis=0, inplace=True)

#split dataframe on companies
stock = {}
for index, row in df.iterrows():
    key = row['Stock']
    if key not in stock:
        stock[key] = pd.DataFrame(columns=df.columns)
        continue
    stock[key] = pd.concat([stock[key], row.to_frame().T], ignore_index=True)

#reshaping dataframe
drop = []
for key in stock:
  stock[key].drop(['Stock'], axis=1, inplace=True)
  company = pd.pivot_table(stock[key], index="Year")

  #min-max scaling
  for col in company.columns:
    try:
      normalized_values = (company[col] - np.min(company[col])) / (np.max(company[col]) - np.min(company[col]))
    except (ZeroDivisionError):
      normalized_values += pd.Series([0.5])
    company.loc[:, col] = normalized_values

  #data cleaning
  company.dropna(axis=0, inplace=True)
  #Mean Imputing
  for col in company.columns:
    company[col] = fill_with_mean(company[col])
  #Median Imputing
  for col in company.columns:
    company[col] = fill_with_median(company[col])

  stock[key] = company
  num_rows = stock[key].shape[0]
  if num_rows:
    pass
  else:
    drop.append(key)

for key in drop:
  stock.pop(key)

In [ ]:
for key in stock:
  try:
    sns.heatmap(stock[key], cmap="gray")
    filename = f"heatmaps/heatmap_{key}.png"
    plt.savefig(filename)
  except (ValueError):
    pass

In [ ]:
!zip -r /content/file.zip /content/heatmaps
files.download("/content/file.zip")

Model Architecture

In [ ]:
ClassifyStock = Sequential([
    Conv2D(64, (58,5), strides=(5,5), activation='relu', input_size=(640,480,3)),
    MaxPool2D((5,5)),

    Conv2D(64, (58,5), strides=(5,5), activation='relu', input_size=(640,480,3)),
    MaxPool2D((5,5)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

Model Compile

In [ ]:
ClassifyStock.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])